Make sure you fill in any place that says `YOUR CODE HERE`. 

---

# Homework 6

*This* is a Python Notebook homework.  It consists of various types of cells: 

* Text: you can read them :-) 
* Code: you should run them, as they may set up the problems that you are asked to solve.
* **Solution:** These are cells where you should enter a solution.  You will see a marker in these cells that indicates where your work should be inserted.  

```
    # YOUR CODE HERE
```    

* Test: These cells contains some tests, and are worth some points.  You should run the cells as a way to debug your code, and to see if you understood the question, and whether the output of your code is produced in the correct format.  The notebook contains both the tests you see, and some secret ones that you cannot see.  This prevents you from using the simple trick of hard-coding the desired output. 

### Questions

There is only one question group in this notebook, namely, implementing the symbolic derivative of an expression with respect to a given variable.  Note that you have to implement this also for the `Power` and `Log` expressions. 

There are other pieces of text called "exercises", but you only have to do those that are explicitly marked with a place in the code for you to write the answer. 

### Working on Your Notebook

To work on your notebook: 

* Click on _File > Save a copy in Drive_ : this will create a copy of this file in your Google Drive; you will find the notebook in your _Colab Notebooks_ folder. 
* Work on that notebook. 

### Submitting Your Notebook

Submit your work as follows: 

* Download the notebook from Colab, clicking on "File > Download .ipynb".
* Upload the resulting file to [this Google form](https://docs.google.com/forms/d/e/1FAIpQLSc21VhzMm8mRFm5c8ttfjhb-D2oSkKjjPmqvLzJYxPbFkaugw/viewform?usp=sf_link).
* **Deadline: Friday November 1, 7pm.**

You can submit multiple times, and the last submittion before the deadline will be used to assign you a grade. 

# Expressions as Classes

Copyright Luca de Alfaro, 2019. 
License: [CC-BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/4.0/).

We will now describe a more sophisticated representation for an expression, based on a hierarchy of classes.  The class **Expr** is the generic class denoting an expression.  It is an _abstract_ class: only its subclasses will be instantiated.  For every operator, such as `+`, there will be a subclass, such as `Plus`.  Variables will correspond to a special subclass, called `V`.  Numerical constants will be just represented by numbers, and not by subclasses of `Expr`. 

## The `Expr` class

The `Expr` class implements the various [methods used to emulate numerical types](https://docs.python.org/3/reference/datamodel.html?#emulating-numeric-types), such as `__add__`, `__sub__`, and so forth.  In this way, we can build an expression simply by writing

    3 * V('x') / 2

Compared with the representation of expressions seen in the previous chapter, this class-based representation offers several advantages.  First, we can build expressions in a natural way as shown above, via the over-riding of the usual arithmetic operators.  Second, if we introduce a new operator, all we need is provide the implementation of the new operator: we do not need to modify the shared code that traverses the tree, and add one more case to a long case-analysis.  In other words, the code is far more modular.  This may seem a small point, but if one were to extend the representation of expressions to involve tensors (matrices) and operations on tensors, as is done in the symbolic representation of expressions used in machine-learning, the number of operators could easily grow to a hundred or more, making a modular approach the only reasonable one. 

### The `eval` method

We endow our class `Expr` with a method `eval`, which given a variable valuation -- a dictionary mapping variable names to values -- tries to compute the expression insofar as possible.  The method `eval` first evaluates the children $c_1, \ldots, c_n$ of the given expression, computing their values $v_1, \ldots, v_n$.  These values $v_i$, for $1 \leq i \leq n$, can be either _symbolic_, if they are an instance of `Expr`, or _numeric_.  They are symbolic if some of the variables in their subexpression was not assigned a value by our variable assignment.  For instance, if we evaluate

    (V('x') + 3) / V('y')

with respect to the assignment `{'x': 2}`, we obtain 

    5 / V('y')

which is still symbolic (an instance of `Expr`).  There are two cases: 

* If all $v_1, \ldots, v_n$ are numeric, `eval` calls the `op` method with arguments $v_1, \ldots, v_n$, to compute the numerical value of the expression.  The method `op` is not implemented in `Expr`, but rather, it is implemented in each operator `Plus`, `Minus`, etc, so as to compute the numerical value according to the operator. 

* If one or more of the results $v_1, \ldots, v_n$ are symbolic, then `eval` returns a new expression, with the same operator as the old one, but with children $v_1, \ldots, v_n$ rather than $c_1, \ldots, c_n$. 

We invite you to study the implementation of `eval` in detail, for the ideas used there will be used also in the implementation of other methods.

In [0]:
class Expr(object):
    """Abstract class representing expressions"""
    
    def __init__(self, *args):
        self.children = list(args)
        self.child_values = None

    def eval(self, valuation=None):
        """Evaluates the value of the expression with respect to a given 
        variable evaluation."""
        # First, we evaluate the children.
        child_values = [c.eval(valuation=valuation) if isinstance(c, Expr) else c
                        for c in self.children]
        # Then, we evaluate the expression itself.
        if any([isinstance(v, Expr) for v in child_values]):
            # Symbolic result.
            return self.__class__(*child_values)
        else:
            # Concrete result.
            return self.op(*child_values)

    def op(self, *args):
        """The op method computes the value of the expression, given the
        numerical value of its subexpressions.  It is not implemented in 
        Expr, but rather, each subclass of Expr should provide its 
        implementation."""
        raise NotImplementedError()

    def __repr__(self):
        """Represents the expression as the name of the class, followed by the 
        children."""
        return "%s(%s)" % (self.__class__.__name__,
                        ', '.join(repr(c) for c in self.children))

    # Expression constructors

    def __add__(self, other):
        return Plus(self, other)

    def __radd__(self, other):
        return Plus(self, other)

    def __sub__(self, other):
        return Minus(self, other)

    def __rsub__(self, other):
        return Minus(other, self)

    def __mul__(self, other):
        return Multiply(self, other)

    def __rmul__(self, other):
        return Multiply(other, self)

    def __truediv__(self, other):
        return Divide(self, other)

    def __rtruediv__(self, other):
        return Divide(other, self)

    def __pow__(self, other):
        return Power(self, other)

    def __rpow__(self, other):
        return Power(other, self)

    def __neg__(self):
        return Negative(self)

Variables are expressions containing only one child, of type string, which is the name of the variable.  So `V("x")` will be a variable with name `"x"`.  The method `eval` of a variable over-rides that of `Expr`. 

In [0]:
class V(Expr):
    """Variable."""
    
    def __init__(self, *args):
        """Variables must be of type string."""
        assert len(args) == 1
        assert isinstance(args[0], str)
        super().__init__(*args)

    def eval(self, valuation=None):
        """If the variable is in the evaluation, returns the
        value of the variable; otherwise, returns the expression."""
        if valuation is not None and self.children[0] in valuation:
            return valuation[self.children[0]]
        else:
            return self

Here are the constructors for the other operators; for them, we just need to provide an implementation for `op`.

In [0]:
class Plus(Expr):
    def op(self, x, y):
        return x + y

class Minus(Expr):
    def op(self, x, y):
        return x - y

class Multiply(Expr):
    def op(self, x, y):
        return x * y

class Divide(Expr):
    def op(self, x, y):
        return x / y

class Power(Expr):
    def op(self, x, y):
        return x ** y

class Negative(Expr):
    def op(self, x):
        return -x

We can build and evaluate expressions quite simply.

In [4]:
e = V('x') + 3
print(e)
print(e.eval())
print(e.eval({'x': 2}))

Plus(V('x'), 3)
Plus(V('x'), 3)
5


In [5]:
e = (V('x') + V('y')) * (2 + V('x'))
print(e.eval())
print(e.eval({'x': 4}))


Multiply(Plus(V('x'), V('y')), Plus(V('x'), 2))
Multiply(Plus(4, V('y')), 6)


### Defining Expression Equality

If we test equality between expressions, we are in for a surprise.

In [6]:
e1 = V('x') + 4
e2 = V('x') + 4
e1 == e2

False

Why is the result False? 

Python knows how to compare objects that belong to its own types.  So you can do comparisons between strings, numbers, tuples, and more, and it all works as expected.  This is why we could check equality of expressions represented as trees: those expression trees are composed entirely of standard Python types, namely, strings, numbers, and tuples. 

However, `Expr`, `V`, etc, are classes we defined, and Python has no idea of what it means for objects of user-defined classes to be equal.  
In this case, Python defaults to considering equal two objects if they are the _same_ object. 
The two expressions _e1_ and _e2_ above are not the same object: they are two distinct objects, which just happen to represent the same expression. 

If we want to have a notion of expression equality that represents our idea that "two expression objects are equal if they represent the same expression", we need to define equality ourselves. 
This can be easily done, by defining an _ _ eq _ _ method.  This method [has the form](https://docs.python.org/3/reference/datamodel.html#object.__eq__):

    def __eq__(self, other):
        ...
        return <True/False>
        
Here, self is the object on which the method is called, and other is another object -- any other object.  Our job is to define when the object self is equal to the object other.  This can be easily done; using again our way of adding methods to existing classes, we write:

In [0]:
def expr_eq(self, other):
    if isinstance(other, Expr):
        # The operators have to be the same
        if self.__class__ != other.__class__:
            return False
        # and their corresponding children need to be equal
        if len(self.children) != len(other.children):
            return false
        for c1, c2 in zip(self.children, other.children):
            if c1 != c2: return False
        return True
    else:
        return False

Expr.__eq__ = expr_eq

Once expression equality is thus defined, we get the expected result when we compare expressions:

In [8]:
e1 = V('x') + 4
e2 = V('x') + 4
e1 == e2

True

Having to define equality "by hand" is very pedantic, but it does give us the flexibility of defining precisely what it means for two expressions to be equal.

We could also have solved the problem in a single stroke as follows:

In [0]:
def equality_as_same_types_and_attributes(self, other):
    return type(self) == type(other) and self.__dict__ == other.__dict__

Expr.__eq__ = equality_as_same_types_and_attributes    

In [10]:
e1 = V('x') + 4
e2 = V('x') + 4
e1 == e2

True

The definition above works because, for an object o, o._ _ dict _ _ () is the dictionary mapping each attribute name to the attribute value.  Dictionaries can be compared, as they are standard Python types, and so comparing object dictionaries yields the expected notion of object equality. 
I certainly wish that Python had chosen the above notion of equality_as_same_types_and_attributes as the default for new classes, rather than the rather useless notion of equality as being the same object. 

We can improve on the above definition by allowing commutativity of `Plus` and `Multiply`:

In [0]:
def commutative_eq(self, other):
    return (isinstance(other, self.__class__) and (
            self.children == other.children or
            (self.children[0] == other.children[1] and 
             self.children[1] == other.children[0])))

Plus.__eq__ = commutative_eq
Multiply.__eq__ = commutative_eq

Note how in the above definition we used `self.__class__` to obtain the class of the object on which the function is called.

In [12]:
try:
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal

     |████████████████████████████████| 163kB 2.8MB/s 


In [0]:
assert_equal(V('x') + 3, 3 + V('x'))
assert_equal(2 * V('x'), V('x') * 2)
assert_not_equal(2 - V('x'), V('x') - 2)

The definition is fine, but note that for deep expression trees, using this notion of equality with commutativity could require exponential time in the size of the tree in the worst case.

### Derivating an expression

We will develop here a method `derivate` such that, for an expression `e`, the method call `e.derivate('x')` returns the derivative of the expression with respect to `V('x')`.  To implement it, we note that to deriva a composite expression with respect to a variable, we need to derivate the subexpressions with respect to that same variable.  Thus, we divide the work as follows: 

* The method `derivate` will first call the method `derivate` for all the subexpressions, that is, the children of the expression, accumulating the results into a list `partials` (so called because these are partial derivatives). 
* Then, the method `derivate` will call `op_derivate`.  This method is an abstract method of `Expr`, and is implemented in each operator (`Plus`, `Minus`, etc).  The method `op_derivate` will compute the derivative of the expression, according to the operator type. 

Let us start by writing `derivate`.


In [0]:
def expr_derivate(self, var):
    """Computes the derivative of the expression with respect to var."""
    partials = [(c.derivate(var) if isinstance(c, Expr) else 0)
                for c in self.children]
    return self.op_derivate(var, partials).eval()

Expr.derivate = expr_derivate

def expr_op_derivate(self, var, partials):
    raise NotImplementedError()

Expr.op_derivate = expr_op_derivate

In the above code, note that we have to take care to call the `derivate` method for a subexpression only if that subexpression has type `Expr`.  If the subexpression does not have type `Expr`, then it is a constant, and its derivative with respect to a variable will be 0. 

The last `eval()` in the above expression is simply an attempt to simplify the resulting expression.  

Let us now write `op_derivate` for `Plus`.  We have:

$$
\frac{\partial}{\partial x} (f + g) \:=\: \frac{\partial f}{\partial x} + \frac{\partial g}{\partial x}
$$

and translating this into code:

In [0]:
def plus_op_derivate(self, var, partials):
    return Plus(partials[0], partials[1])

Plus.op_derivate = plus_op_derivate

Before we can test the code, we need to have some variables. What is the derivative of `V('x')` with respect to a variable?  If the variable is `x`, that is, the same variable appearing in the `V` constructor, then the derivative is 1, otherwise, it is 0.  This because for variables, 

$$
\frac{\partial x}{\partial x} = 1 \qquad \frac{\partial x}{\partial y} = 0
\; .
$$

We will let you translate this into code; it's a one-liner. 

In [0]:
### Definition of `V.derivate`

def variable_derivate(self, var):
    # YOUR CODE HERE
    if self.children[0] == var:
      return 1 
    else:
      return 0

V.derivate = variable_derivate

In [17]:
e = V('x') + 3
print(e.derivate('x'))
assert_equal(e.derivate('x'), 1)
assert_equal(e.derivate('y'), 0)

1


We now let you define `op_derivate` for the other operators.  If you need to refresh your mind, [you can consult the Wikipedia page on derivatives](https://en.wikipedia.org/wiki/Derivative).

**Exercise:** define `op_derivate` for `Minus` and `Multiply`.

In [0]:
### `Minus.op_derivate`

def minus_op_derivate(self, var, partials):
    # YOUR CODE HERE
    return Minus(partials[0], partials[1])

Minus.op_derivate = minus_op_derivate

In [0]:
### Tests for `Minus.op_derivate`

e = V('x') - 4
assert_equal(e.derivate('x'), 1)

e = 4 - V('x')
assert_equal(e.derivate('x'), -1)


In [0]:
### `Multiply.op_derivate`

def multiply_op_derivate(self, var, partials):
    # YOUR CODE HERE
    return Plus(Multiply(partials[0], self.children[1]), (Multiply(partials[1], self.children[0])))

Multiply.op_derivate = multiply_op_derivate

In [0]:
### Tests for `Multiply.op_derivate`

e = V('x') * V('y')
assert_equal(e.derivate('x').eval(dict(x=3, y=2)), 2)


Why do we use the .eval() in the above assertion?  Because the problem is that, if you implement derivatives in the natural way, the expression you obtain is not simplified.  For instance, this is what we get:

In [22]:
e.derivate('x')

Plus(Multiply(1, V('y')), Multiply(0, V('x')))

Obviously, `1 * V('y')` should be simplified to `V('y')`, and `0 * V('x')` should be simplified to `0`.  We will work on carrying out these 0-1 simplifications later.  For the moment, let's move to division. 

**Exercise:** Define the operator `op_derivate` for `Divide`.

In [0]:
### `Divide.op_derivate`

def divide_op_derivate(self, var, partials):
    # YOUR CODE HERE
    return(Divide(Minus(Multiply(partials[0], self.children[1]), Multiply(partials[1], self.children[0])), Power(self.children[1], 2)))

Divide.op_derivate = divide_op_derivate

In [0]:
### Tests for `Divide.op_derivate`

e = V('x') / V('y')
assert_equal(e.derivate('x').eval(dict(x=3, y=2)), 0.5)
assert_equal(e.derivate('y').eval(dict(x=3, y=2)), -3 / 4)


### Of Power, Logarithms, And Closure. 

> _Once many years ago in Italy I saw an old PC that was a clone of the famous first [IBM PCs](https://en.wikipedia.org/wiki/IBM_Personal_Computer).  The cloners had made an effort to translate it into italian, but the clone itself was manufactured in Asia.  The Power button was labeled "Potere", which translates as "Might".  I imagined someone with lusty eyes pressing on the "Might" button, in a sorcerer's apprentice kind of way.  I did not imagine how accurate that could be._


Let's move on to Power.  We have: 

$$
\begin{align}
 \frac{\partial}{\partial x} f^g & = \frac{\partial}{\partial x} e^{g \log f} \\[1ex]
& = e^{g \log f} \left[\frac{g}{f} \frac{\partial f}{\partial x}
                        + \frac{\partial g}{\partial x} \log f \right] \\[1ex]
& = f^g \left[\frac{g}{f} \frac{\partial f}{\partial x}
                        + \frac{\partial g}{\partial x} \log f \right] \; .
\end{align}
$$

This is precisely why we need `Logarithm` to be one of our operators.  Without it, the set of symbolic expressions would not be closed with respect to symbolic differentiation.  How can we add the logarithm?  Using 

$$
 \frac{\partial}{\partial x} \log f = \frac{1}{f} \, 
 \frac{\partial}{\partial x} f \; ,
$$

we define a new class:

In [0]:
import math

class Logarithm(Expr):

    def op(self, x):
        return math.log(x)

    def op_derivate(self, var, partials):
        return Multiply(
            Divide(1, self.children[0]), partials[0]
        )

Note that the only way to construct an expression containing a logarithm is to use the `Logarithm` constructor, but this is just fine.  Let us give this a try:

In [26]:
e = Logarithm(V('x'))
print(e.eval(dict(x=1)))
print(e.derivate('x'))

0.0
Multiply(Divide(1, V('x')), 1)


Simplifications aside, this is fine.  We can then tackle `Power`, and `Negative`.

**Exercise:** define the operator `op_derivate` for `Power`. 

In [0]:
### `Power.op_derivate`

import math

def power_op_derivate(self, var, partials):
    # YOUR CODE HERE
    return(Multiply(Power(self.children[0], self.children[1]), Plus(Multiply(Divide(self.children[1], self.children[0]), partials[0]), Multiply(partials[1], Logarithm(self.children[0])))))

Power.op_derivate = power_op_derivate

In [0]:
### Tests for `Power.op_derivate`

e = 3 ** V('x')
assert_almost_equal(e.derivate('x').eval(dict(x=4)), 
                    math.log(3) * (3 ** 4), places=2)

e = V('x') ** 2.8
assert_almost_equal(e.derivate('x').eval(dict(x=3)), 2.8 * 3 ** 1.8, places=2)
